In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05067729949951172
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 5.475441545659295
200000 8.435336417953872
300000 8.973391304647741
400000 10.600312270709853
500000 12.16051949971171
600000 13.594808690219542
700000 13.133960422689468
800000 11.786562390662866
900000 14.457161198337886
1000000 16.2390894627999
1100000 16.436776945196264
1200000 9.374470932494269
1300000 22.891954116381992
1400000 22.457193778390515
1500000 22.265717248277014
1600000 22.34039258884254
1700000 23.613900157602206
1800000 22.27456170158885
1900000 22.406986775922576
2000000 22.266368955632863
2100000 22.874179135745326
2200000 22.290120434599054
2300000 5.8248973998085525
2400000 25.352591360253967
2500000 27.79778356430402
2600000 28.057411671579256
2700000 27.475375008197293
2800000 27.69247396067678
2900000 27.89114410024218
3000000 25.9

29400000 21.203980927358703
29500000 7.54604128298065
29600000 21.50132203236464
29700000 21.852117874363074
29800000 20.938109278024736
29900000 20.819459908834155
30000000 21.244623646542806
30100000 20.14008534100591
30200000 20.514447492065294
30300000 19.259043962010452
30400000 5.690482869336456
30500000 20.39558500520075
30600000 19.067626273140608
30700000 20.55831436950757
30800000 19.990686322718815
30900000 21.019511435545823
31000000 20.610006281101207
31100000 20.76793172975686
31200000 21.729674966500536
31300000 8.922958796451061
31400000 21.623236321855764
31500000 20.70871937832694
31600000 21.738623634189874
31700000 18.012364079703183
31800000 10.786279617049138
31900000 21.609427244685808
32000000 19.908979916407397
32100000 20.913180943981853
32200000 19.419884499493715
32300000 19.539991846339568
32400000 20.0648158478139
32500000 18.01640529157108
32600000 12.09307769205061
32700000 13.71852672584462
32800000 18.31948793411971
32900000 19.01103315681548
33000000 

59100000 16.78104414375747
59200000 17.56387810016683
59300000 15.813832459381238
59400000 16.12871616697137
59500000 16.407120264160476
59600000 7.345765598379334
59700000 15.352208205571134
59800000 15.913917509049575
59900000 16.541583144761596
60000000 16.182744231594125
60100000 16.507928300423863
60200000 17.953142318448187
60300000 7.474158574051549
60400000 19.08659945882639
60500000 17.936257327392372
60600000 18.05255455618871
60700000 18.82311866651683
60800000 18.41983453397712
60900000 19.402196272370567
61000000 18.04632895412177
61100000 19.373174533954753
61200000 8.171456357939432
61300000 16.12695696351098
61400000 18.044733645104028
61500000 16.292884014424704
61600000 17.540962854498286
61700000 16.287432017793247
61800000 14.729750361990622
61900000 8.441680017580836
62000000 16.166410401696005
62100000 17.154725527439037
62200000 15.944439563210777
62300000 17.185357635872368
62400000 17.395465016815486
62500000 9.41095115055588
62600000 16.78173842531106
62700000

88800000 9.961036197215233
88900000 10.405650444761843
89000000 14.954581512560207
89100000 14.628397311217832
89200000 14.99768764017121
89300000 6.437803245717744
89400000 15.50862667464059
89500000 15.243851273563742
89600000 16.08697493134236
89700000 16.115335936011885
89800000 15.821678767543107
89900000 15.994798023295239
90000000 16.054078177055324
90100000 15.728582060760965
90200000 15.917984259176638
90300000 16.0483560467093
90400000 15.144195173400988
90500000 14.685342883825015
90600000 11.96335088318221
90700000 8.255252532583475
90800000 13.947234673226218
90900000 13.976123640028437
91000000 14.573714593614103
91100000 4.15966627371197
91200000 15.005715607267874
91300000 14.434337440755783
91400000 13.861303793627904
91500000 8.047791857973328
91600000 12.540704060481369
91700000 14.49772479058556
91800000 15.092514623543584
91900000 15.285886957319516
92000000 16.080707122397044
92100000 15.24621312840362
92200000 16.10343447025808
92300000 15.367387958214424
9240000

117800000 8.942351256069458
117900000 8.750645505342252
118000000 10.537287784268756
118100000 5.802731898460444
118200000 12.174113264058978
118300000 5.523853148353407
118400000 12.944058795298606
118500000 12.312177334569904
118600000 12.816526647074998
118700000 12.539526523026433
118800000 12.93862923261273
118900000 12.906988910828433
119000000 13.234888809838427
119100000 14.152008628359146
119200000 13.948360741729877
119300000 14.18428789786844
119400000 13.929613956779027
119500000 12.271409101104439
119600000 12.61385536870023
119700000 12.346906963446921
119800000 12.314369157691221
119900000 12.319351790608739
120000000 12.118613651833808
120100000 5.5492939331685
120200000 11.173927806815326
120300000 4.896485532065914
120400000 11.149189621017486
120500000 7.351719423192495
120600000 10.904764993715375
120700000 12.073303894439976
120800000 12.533147860040375
120900000 11.887790184685171
121000000 12.321600492013772
121100000 12.19167845648005
121200000 12.40890047931277

...

175500000 9.39611122247551
175600000 8.957064257311304
175700000 8.83076960679562
175800000 8.501870504354768
175900000 8.878506687919149
176000000 8.608950970696538
176100000 8.37770273340388
176200000 8.563880912762492
176300000 3.7299799629322816
176400000 8.793388408091545
176500000 3.705727985644162
176600000 7.401241845921616
176700000 5.475451866633214
176800000 6.017355308773618
176900000 8.979008342779187
177000000 8.20513737427977
177100000 8.79586561023116
177200000 8.533074080914727
177300000 8.720164833191987
177400000 9.185751342224766
177500000 9.370488961579708
177600000 9.73429984088511
177700000 9.462962262982549
177800000 9.259071349370963
177900000 8.948863946657976
178000000 8.763312409520298
178100000 8.849176739253634
178200000 8.584582849664791
178300000 8.57089444255049
178400000 8.413637193708212
178500000 5.235553068010933
178600000 7.505115830209842
178700000 7.02059303396832
178800000 4.148300492916369
178900000 8.317925602852315
179000000 3.699795419172793

204800000 7.2841058069271645
204900000 7.114481874642624
205000000 6.925026048890591
205100000 6.759234848328558
205200000 6.592465175107965
205300000 2.7692455984165374
205400000 6.726353159109318
205500000 6.423955682541732
205600000 6.605214709263354
205700000 4.463710439704021
205800000 4.560553748844435
205900000 6.544809163339942
206000000 6.715805943290151
206100000 6.6079851215529875
206200000 2.393248526409666
206300000 6.735133760699097
206400000 6.71015797188835
206500000 7.1462065059073865
206600000 7.147728207391785
206700000 7.363345290019976
206800000 7.236835233734816
206900000 7.109718307220158
207000000 7.424761772634378
207100000 7.195176733733445
207200000 6.716660627019136
207300000 6.847301572960141
207400000 6.472336297714452
207500000 4.069474783154554
207600000 5.353144633153153
207700000 6.396802994661927
207800000 6.29884644652117
207900000 6.493246378010253
208000000 1.8468784452578624
208100000 6.6150857124072475
208200000 6.3112052147920465
208300000 6.369

234000000 4.736751447623004
234100000 4.885727476640999
234200000 4.827412481753253
234300000 1.9824082512055536
234400000 4.942587170226852
234500000 4.371712705326286
234600000 4.411441886315717
234700000 4.265602738808066
234800000 4.328600082082788
234900000 3.8054681485254305
235000000 2.0661137652570707
235100000 4.381758097052101
235200000 4.098472270697319
235300000 4.256058719506055
235400000 4.4628654459813735
235500000 4.499317066104796
235600000 3.0434191637487413
235700000 3.380439925646431
235800000 4.925490746165266
235900000 4.714520969943751
236000000 4.756101884436339
236100000 5.019955146793578
236200000 4.5938531535837255
236300000 4.974341630409853
236400000 4.647578907257849
236500000 2.946210948410511
236600000 3.85955213450646
236700000 4.428281575533125
236800000 4.496721594621469
236900000 4.1195659534811675
237000000 4.341860991584503
237100000 4.171498135598013
237200000 1.8737445159629549
237300000 3.9184264437176353
237400000 4.171350673186003
237500000 4.

262900000 2.3875982337357504
263000000 2.783088591787739
263100000 2.772642349887467
263200000 2.6959313083496688
263300000 1.1255432015841438
263400000 2.85286723940997
263500000 2.600683192376435
263600000 2.7243493783409463
263700000 2.65321278093838
263800000 2.5048577363999884
263900000 2.632065750815196
264000000 2.4100160300468505
264100000 2.555535773388763
264200000 0.7453114849589235
264300000 2.5333737099245623
264400000 2.507681030358821
264500000 2.513055780262103
264600000 2.717688772628794
264700000 2.5632416317489644
264800000 2.6899781910270586
264900000 2.6454382795037366
265000000 2.7020144358020923
265100000 0.931640174451434
265200000 2.583354185828884
265300000 2.7974191908352077
265400000 2.5083094953483047
265500000 1.6284067892245617
265600000 2.1084491148239075
265700000 2.477734361776766
265800000 2.637399993471824
265900000 2.3466322321858404
266000000 2.581168522436708
266100000 2.3643547473404904
266200000 2.3762005656249823
266300000 2.280959587981022
266

291400000 0.4875141575747431
291500000 0.45850796826796736
291600000 0.4633666687952479
291700000 0.4672793683958749
291800000 0.43778360503803027
291900000 0.43491558964164717
292000000 0.4159525898308158
292100000 0.41717579387540615
292200000 0.3400236010774308
292300000 0.08564633827254176
292400000 0.30848682436988417
292500000 0.3026908861737351
292600000 0.30071757697921986
292700000 0.2747138700580663
292800000 0.2779149386387931
292900000 0.26652857967994614
293000000 0.24409750849627457
293100000 0.23953442297164929
293200000 0.2374585271333158
293300000 0.2201196678716971
293400000 0.09641604741003117
293500000 0.19531637730884552
293600000 0.2046655632396274
293700000 0.20025165996655492
293800000 0.19287488723947604
293900000 0.18788712936100033
294000000 0.19168909989141425
294100000 0.16891627663931252
294200000 0.1552306377763682
294300000 0.1660140333422588
294400000 0.14749380600030224
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.